In [1]:
!pip -q install elasticsearch==8.15.1 sentence-transformers \
  langchain langchain-core langchain-community tiktoken


In [44]:
from elasticsearch import Elasticsearch
import getpass

ES_HOST = input("Elastic Cloud endpoint (https://<endpoint>:443): ").strip()
## https://my-elasticsearch-project-b71e61.es.us-central1.gcp.elastic.cloud:443   <- 입력 호스트주소
ES_API_KEY = getpass.getpass("Paste your ES API Key: ")
## api_key  TVk4anRwb0JjYlhwVURvWl96a0w6ZVBlbnBmY2x4SkF3R0picjkyekpvUQ==     <- api key 입력
es = Elasticsearch(ES_HOST, api_key=ES_API_KEY, request_timeout=60)
es.info()


Elastic Cloud endpoint (https://<endpoint>:443): https://my-elasticsearch-project-b71e61.es.us-central1.gcp.elastic.cloud:443
Paste your ES API Key: ··········


ObjectApiResponse({'name': 'serverless', 'cluster_name': 'b71e61fdebf44b8e91f738f3739d9605', 'cluster_uuid': 'QkSuMQzMSv6OryLeqnGh7w', 'version': {'number': '8.11.0', 'build_flavor': 'serverless', 'build_type': 'docker', 'build_hash': '00000000', 'build_date': '2023-10-31', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '8.11.0', 'minimum_index_compatibility_version': '8.11.0'}, 'tagline': 'You Know, for Search'})

In [3]:
INDEX_HYB = "realestate_hybrid"   # 네가 사용 중인 이름
print("exists:", es.indices.exists(index=INDEX_HYB))

# 문서수
try:
    print(es.count(index=INDEX_HYB))
except Exception as e:
    print("count error:", e)

# embedding 차원 확인
m = es.indices.get_mapping(index=INDEX_HYB)
print(m[INDEX_HYB]["mappings"]["properties"]["embedding"])


exists: True
{'count': 30575, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}}
{'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'bbq_hnsw', 'm': 16, 'ef_construction': 100, 'rescore_vector': {'oversample': 3.0}}}


In [4]:
from sentence_transformers import SentenceTransformer
import torch

EMB_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"
sbert = SentenceTransformer(EMB_MODEL_NAME, device=device)

def embed_query(q: str):
    return sbert.encode([q], normalize_embeddings=True, show_progress_bar=False)[0].tolist()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# === D) 하이브리드 검색 (정밀 필터 + RRF, ES 미지원시 파이썬 RRF 폴백) ===
import re

BRANDS = ["래미안","자이","푸르지오","e편한세상","힐스테이트","롯데캐슬","아이파크",
          "더샵","호반","중흥","트리마제","서희","우성","센트럴","리버뷰"]

def extract_hints(q: str):
    # 면적 힌트(59, 84 등)
    area = None
    m = re.search(r'(\d{2,3})\s*(?:㎡|m2|m²|형)?', q)
    if m:
        v = int(m.group(1))
        if 15 <= v <= 200:
            area = float(v)
    # 브랜드 힌트
    brand = next((b for b in BRANDS if b in q), None)
    return area, brand

def _bm25_query(q: str, filters, brand=None):
    should = [
        {"match": {"apt_name": {"query": q, "operator": "and", "boost": 2.0}}},
        {"match_phrase_prefix": {"apt_name": {"query": q, "boost": 1.5}}}
    ]
    if brand:
        should.append({"match_phrase": {"apt_name": {"query": brand, "boost": 3.0}}})
    return {
        "query": {
            "bool": {
                "should": should,
                "filter": filters or [],
                "minimum_should_match": 1
            }
        }
    }

def _knn_query(q_vec, filters, k=80, num_candidates=1000):
    knn = {
        "field": "embedding",
        "query_vector": q_vec,
        "k": k,
        "num_candidates": num_candidates
    }
    if filters:
        knn["filter"] = {"bool": {"filter": filters}}
    return knn

def hybrid_search(q: str, gu_filter=None, dong_filter=None, area_center=None, tol=1.0, size=10):
    # 1) 힌트 추출
    area_hint, brand_hint = extract_hints(q)
    if area_center is None:
        area_center = area_hint

    # 2) 쿼리 임베딩 (앞에서 정의한 embed_query 사용)
    q_vec = embed_query(q)

    # 3) 필터 강화
    filters = []
    if gu_filter:
        filters.append({"terms": {"gu": gu_filter if isinstance(gu_filter, list) else [gu_filter]}})
    if dong_filter:
        filters.append({"terms": {"dong": dong_filter if isinstance(dong_filter, list) else [dong_filter]}})
    if area_center is not None:
        filters.append({"range": {"area_m2": {"gte": area_center - tol, "lte": area_center + tol}}})

    # 4) ES RRF 지원 시도
    bm25 = _bm25_query(q, filters, brand=brand_hint)
    knn  = _knn_query(q_vec, filters)

    body_rrf = {
        "_source": ["gu","dong","apt_name","deal_date","area_m2","price_krw","year_built"],
        "size": size,
        "query": bm25["query"],        # BM25
        "knn": knn,                    # kNN
        "rank": {"rrf": {"window_size": 50, "rank_constant": 60}}
    }
    try:
        res = es.search(index=INDEX_HYB, body=body_rrf)
        return [h["_source"] for h in res["hits"]["hits"]]
    except Exception:
        pass  # 폴백 진행

    # 5) 폴백: BM25, kNN 각각 조회 → 파이썬 RRF 결합
    res_bm25 = es.search(index=INDEX_HYB, body={**bm25, "_source":["gu","dong","apt_name","deal_date","area_m2","price_krw","year_built"], "size":120})
    res_knn  = es.search(index=INDEX_HYB, body={"knn": _knn_query(q_vec, filters, k=120, num_candidates=1000),
                                                "_source":["gu","dong","apt_name","deal_date","area_m2","price_krw","year_built"]})
    bm25_hits, knn_hits = res_bm25["hits"]["hits"], res_knn["hits"]["hits"]

    k_const, ranks = 60, {}
    for r, h in enumerate(bm25_hits, 1):
        _id = h["_id"]; ranks.setdefault(_id, {"score":0, "src":h}); ranks[_id]["score"] += 1/(k_const+r)
    for r, h in enumerate(knn_hits, 1):
        _id = h["_id"]; ranks.setdefault(_id, {"score":0, "src":h}); ranks[_id]["score"] += 1/(k_const+r)

    # 추가 재랭크: 브랜드 보너스, 면적 편차 패널티
    def rerank(item):
        src, score = item["src"]["_source"], item["score"]
        if brand_hint and brand_hint in str(src.get("apt_name","")):
            score += 0.05
        if area_center is not None and src.get("area_m2") is not None:
            score -= 0.01 * abs(src["area_m2"] - area_center)
        return score

    fused = sorted(ranks.values(), key=rerank, reverse=True)[:size]
    return [x["src"]["_source"] for x in fused]


In [6]:
# 인덱스/문서/매핑
es.indices.exists(index=INDEX_HYB)
es.count(index=INDEX_HYB)
es.indices.get_mapping(index=INDEX_HYB)[INDEX_HYB]["mappings"]["properties"]["embedding"]

# 임베딩 크기
len(embed_query("테스트"))  # 384

# 간단 검색
from pprint import pprint
rows = hybrid_search("공덕 래미안 59", gu_filter="마포구", size=10)
pprint(rows[:3])


[{'apt_name': '래미안공덕4차',
  'area_m2': 59.9,
  'deal_date': '2025-07-16',
  'dong': '공덕동',
  'gu': '마포구',
  'price_krw': 1550000000,
  'year_built': 2005},
 {'apt_name': '래미안공덕5차',
  'area_m2': 59.959,
  'deal_date': '2025-09-21',
  'dong': '공덕동',
  'gu': '마포구',
  'price_krw': 1740000000,
  'year_built': 2011},
 {'apt_name': '래미안공덕4차',
  'area_m2': 59.9,
  'deal_date': '2025-05-31',
  'dong': '공덕동',
  'gu': '마포구',
  'price_krw': 1535000000,
  'year_built': 2005}]


In [7]:
def pretty(rows, cols=("gu","dong","apt_name","deal_date","area_m2","year_built","price_krw")):
    for r in rows:
        line=[]
        for c in cols:
            v=r.get(c)
            if c=="price_krw" and v is not None: v=f"{int(v):,}원"
            if c=="area_m2"  and v is not None: v=f"{float(v):.1f}"
            line.append(f"{c}: {v}")
        print("{ "+", ".join(line)+" }")
    print(f"총 {len(rows)}건\n")


In [8]:
def loan_payment(principal_krw: int, annual_rate_pct: float, years: int) -> dict:
    n = int(years)*12
    r = (annual_rate_pct/100)/12
    pay = principal_krw/n if r==0 else principal_krw*(r*(1+r)**n)/((1+r)**n-1)
    return {"monthly_payment": int(round(pay)), "months": n, "total_payment": int(round(pay*n))}


In [9]:
def search_demo(query, gu_list, area_center=None, tol=1.2, brand_required=False, price_max=None, size=10):
    rows = hybrid_search(query, gu_filter=gu_list, area_center=area_center, tol=tol, size=size*3)
    # 브랜드 필터(쿼리에 포함된 브랜드가 있을 때만 강제)
    brands = ["래미안","자이","푸르지오","e편한세상","힐스테이트","롯데캐슬","아이파크","더샵","호반","중흥","트리마제","서희","우성","센트럴","리버뷰"]
    brand = next((b for b in brands if b in query), None)
    if brand_required and brand:
        rows = [r for r in rows if brand in str(r.get("apt_name",""))]
    if price_max is not None:
        rows = [r for r in rows if r.get("price_krw") and r["price_krw"]<=price_max]
    if area_center is not None:
        rows = [r for r in rows if r.get("area_m2") is not None and (area_center-tol)<=float(r["area_m2"])<=(area_center+tol)]
    rows.sort(key=lambda x:(x.get("deal_date",""), -(x.get("price_krw") or 0)), reverse=True)
    return rows[:size]


In [29]:
def get_all_gus(index=INDEX_HYB):
    body = {"size":0, "aggs":{"g":{"terms":{"field":"gu","size":100}}}}
    res = es.search(index=index, body=body)
    return [b["key"] for b in res["aggregations"]["g"]["buckets"]]

ALL_GUS = get_all_gus()  # 예: ['강남구','강동구', ... '송파구']


In [38]:
def top_price_year(year, gu, area_center=59.0, tol=1.0, size=3):
    body={
      "query":{"bool":{"must":[
        {"term":{"gu":gu}},
        {"range":{"deal_date":{"gte":f"{year}-01-01","lte":f"{year}-12-31"}}},
        {"range":{"area_m2":{"gte":area_center-tol,"lte":area_center+tol}}}
      ]}},
      "sort":[{"price_krw":"desc"}],
      "_source":["gu","dong","apt_name","deal_date","area_m2","price_krw"],
      "size": size
    }
    res = es.search(index=INDEX, body=body)
    return [h["_source"] for h in res["hits"]["hits"]]

In [34]:
from collections import defaultdict

def balanced_top_k(rows, per_gu=3, max_total=30):
    out, take = [], defaultdict(int)
    for r in rows:
        g = r.get("gu")
        if g and take[g] < per_gu:
            out.append(r); take[g]+=1
        if len(out) >= max_total:
            break
    return out


In [31]:
from collections import defaultdict

def balanced_top_k(rows, per_gu=3, max_total=30):
    out, take = [], defaultdict(int)
    for r in rows:
        g = r.get("gu")
        if g and take[g] < per_gu:
            out.append(r); take[g]+=1
        if len(out) >= max_total:
            break
    return out


In [10]:
# 최근 3개월 평균 (구 단위)
def avg_price_last_3m(gu, area_center=59.0, tol=1.0):
    body = {
      "query": {"bool": {"must": [
        {"term": {"gu": gu}},
        {"range": {"deal_date": {"gte": "now-3M/M", "lte": "now"}}},
        {"range": {"area_m2": {"gte": area_center - tol, "lte": area_center + tol}}}
      ]}},
      "aggs": {
        "avg_price": {"avg": {"field": "price_krw"}},
        "count": {"value_count": {"field": "price_krw"}}
      },
      "size": 0
    }
    res = es.search(index=INDEX_HYB, body=body)
    return res["aggregations"]["avg_price"]["value"], int(res["aggregations"]["count"]["value"])

# 최근 3개월 평균 (구+동 단위)
def avg_price_last_3m_dong(gu, dong, area_center=59.0, tol=1.0):
    body = {
      "query": {"bool": {"must": [
        {"term": {"gu": gu}},
        {"term": {"dong": dong}},
        {"range": {"deal_date": {"gte": "now-3M/M", "lte": "now"}}},
        {"range": {"area_m2": {"gte": area_center - tol, "lte": area_center + tol}}}
      ]}},
      "aggs": {
        "avg_price": {"avg": {"field": "price_krw"}},
        "count": {"value_count": {"field": "price_krw"}}
      },
      "size": 0
    }
    res = es.search(index=INDEX_HYB, body=body)
    return res["aggregations"]["avg_price"]["value"], int(res["aggregations"]["count"]["value"])


In [26]:
# 인덱스에 문서수/기간/가격 분포 대략 보기
print("docs:", es.count(index=INDEX_HYB))

# 최근 10년 컷
from datetime import datetime
cut = datetime.now().year - 10

# 최근 10년 준공 문서 수
q_recent = {
  "query": {"bool": {"filter": [{"range":{"year_built":{"gte": cut}}}]}},
  "size": 0, "aggs": {
    "areas": {"histogram":{"field":"area_m2","interval":1}},
    "prices": {"histogram":{"field":"price_krw","interval":100_000_000}},
    "by_gu": {"terms":{"field":"gu","size":30}}
  }
}
res = es.search(index=INDEX_HYB, body=q_recent)
print("최근10년 문서수:", res["hits"]["total"]["value"])
print("구 Top:", [b["key"] for b in res["aggregations"]["by_gu"]["buckets"][:10]])



docs: {'count': 30575, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}}
최근10년 문서수: 4360
구 Top: ['송파구', '마포구', '노원구']


In [27]:
def search_by_filters(gu_list=None, area_center=None, tol=1.5,
                      price_max=None, year_built_min=None,
                      size=50):
    must = []
    filters = []
    if gu_list:
        filters.append({"terms":{"gu": gu_list}})
    if area_center is not None:
        filters.append({"range":{"area_m2":{"gte": area_center - tol, "lte": area_center + tol}}})
    if price_max is not None:
        filters.append({"range":{"price_krw":{"lte": price_max}}})
    if year_built_min is not None:
        filters.append({"range":{"year_built":{"gte": year_built_min}}})
    body = {
        "query":{"bool":{"must": must, "filter": filters}},
        "_source":["gu","dong","apt_name","deal_date","area_m2","year_built","price_krw"],
        "sort":[{"deal_date":"desc"},{"price_krw":"asc"}],
        "size": size
    }
    res = es.search(index=INDEX_HYB, body=body)
    return [h["_source"] for h in res["hits"]["hits"]]

# (A) 서울 59㎡ ±1.8, 신축 10년, 10억 이하
from datetime import datetime
cut = datetime.now().year - 10
rows = search_by_filters(
    gu_list=None,                # 전체 서울(인덱스 전체)에서
    area_center=59, tol=1.8,     # 면적 폭 조금 넓힘
    price_max=1_000_000_000,     # 10억
    year_built_min=cut,          # 최근 10년
    size=100
)
pretty(rows[:20])


{ gu: 노원구, dong: 상계동, apt_name: 포레나노원, deal_date: 2025-10-19, area_m2: 60.0, year_built: 2020, price_krw: 940,000,000원 }
{ gu: 노원구, dong: 중계동, apt_name: 현대(6차), deal_date: 2025-10-18, area_m2: 59.8, year_built: 2018, price_krw: 550,000,000원 }
{ gu: 노원구, dong: 상계동, apt_name: 노원센트럴푸르지오, deal_date: 2025-10-18, area_m2: 60.0, year_built: 2021, price_krw: 845,000,000원 }
{ gu: 노원구, dong: 상계동, apt_name: 포레나노원, deal_date: 2025-10-18, area_m2: 59.9, year_built: 2020, price_krw: 930,000,000원 }
{ gu: 노원구, dong: 상계동, apt_name: 포레나노원, deal_date: 2025-10-18, area_m2: 60.0, year_built: 2020, price_krw: 946,000,000원 }
{ gu: 노원구, dong: 월계동, apt_name: 월계센트럴아이파크, deal_date: 2025-10-17, area_m2: 60.0, year_built: 2020, price_krw: 610,000,000원 }
{ gu: 노원구, dong: 상계동, apt_name: 노원롯데캐슬시그니처, deal_date: 2025-10-17, area_m2: 60.0, year_built: 2023, price_krw: 900,000,000원 }
{ gu: 노원구, dong: 월계동, apt_name: 녹천역두산위브아파트, deal_date: 2025-10-15, area_m2: 59.8, year_built: 2017, price_krw: 635,000,000원 }
{ gu: 노원구, do

In [36]:
# 전 구 대상으로 59㎡ 후보 넓게 뽑음
rows = search_demo(
    "59",                     # 브랜드 없음
    gu_list=ALL_GUS,          # ← ①에서 만든 전 구 리스트
    area_center=59, tol=1.8,  # 면적 여유
    brand_required=False,
    size=200
)
# 10억 이하 후처리
rows = [r for r in rows if r.get("price_krw") and r["price_krw"] <= 1_000_000_000]
# 구별로 고르게 정리(② 함수 사용)
rows_bal = balanced_top_k(rows, per_gu=3, max_total=30)
pretty(rows_bal)


{ gu: 노원구, dong: 하계동, apt_name: 학여울청구, deal_date: 2025-09-20, area_m2: 59.4, year_built: 1999, price_krw: 640,000,000원 }
{ gu: 노원구, dong: 하계동, apt_name: 학여울청구, deal_date: 2025-09-12, area_m2: 59.4, year_built: 1999, price_krw: 620,000,000원 }
{ gu: 노원구, dong: 하계동, apt_name: 학여울청구, deal_date: 2025-08-27, area_m2: 59.4, year_built: 1999, price_krw: 627,000,000원 }

총 3건



In [37]:
print("=== (Q1) 서울 10억 이하 59형 실거래가 ===")
from datetime import datetime
cut = datetime.now().year - 10

rows = search_by_filters(
    gu_list=None,             # 전국(인덱스 전체)
    area_center=59, tol=1.8,  # 면적 폭
    price_max=1_000_000_000,  # 10억 이하
    year_built_min=cut,       # 최근 10년
    size=300
)
rows = balanced_top_k(rows, per_gu=3, max_total=30)  # 보기 좋게
pretty(rows)

print("\n=== (Q2) 송파구 신천동 84㎡ 최근 3개월 평균 ===")
avg, n = avg_price_last_3m_dong("송파구", "신천동", area_center=84, tol=1.5)
print(f"신천동 84㎡ 최근3개월 평균: {0 if avg is None else int(avg):,}원 (표본 {n}건)")

print("\n=== (Q3) 상환 시뮬레이션 (현금 2억, 매매 9.5억, 30년 3.8%) ===")
price = 950_000_000
cash  = 200_000_000
loan  = price - cash
res   = loan_payment(loan, 3.8, 30)
print(f"대출원금: {loan:,}원 | 월 상환: {res['monthly_payment']:,}원 | 총 {res['months']}개월")


=== (Q1) 서울 10억 이하 59형 실거래가 ===
{ gu: 노원구, dong: 상계동, apt_name: 포레나노원, deal_date: 2025-10-19, area_m2: 60.0, year_built: 2020, price_krw: 940,000,000원 }
{ gu: 노원구, dong: 중계동, apt_name: 현대(6차), deal_date: 2025-10-18, area_m2: 59.8, year_built: 2018, price_krw: 550,000,000원 }
{ gu: 노원구, dong: 상계동, apt_name: 노원센트럴푸르지오, deal_date: 2025-10-18, area_m2: 60.0, year_built: 2021, price_krw: 845,000,000원 }
{ gu: 송파구, dong: 가락동, apt_name: 강남팰리스, deal_date: 2025-10-14, area_m2: 57.6, year_built: 2022, price_krw: 700,000,000원 }
{ gu: 마포구, dong: 망원동, apt_name: 뉴성신아파트, deal_date: 2025-09-26, area_m2: 58.8, year_built: 2021, price_krw: 710,000,000원 }
{ gu: 송파구, dong: 송파동, apt_name: 한영해시안아파트, deal_date: 2025-09-26, area_m2: 58.8, year_built: 2019, price_krw: 880,000,000원 }
{ gu: 마포구, dong: 망원동, apt_name: 솔루체, deal_date: 2025-09-18, area_m2: 57.9, year_built: 2021, price_krw: 699,000,000원 }
{ gu: 송파구, dong: 가락동, apt_name: 강남팰리스, deal_date: 2025-09-08, area_m2: 57.6, year_built: 2022, price_krw: 700,000,

In [33]:
# 공덕 래미안 59 (브랜드 엄격 + 동/구 필터)
base = search_by_filters(gu_list=["마포구"], area_center=59, tol=1.5, size=300)
rows = [r for r in base if "공덕" in str(r.get("dong","")) and "래미안" in str(r.get("apt_name",""))]
pretty(rows[:30])

# 0건이면 tol 확대
if not rows:
    base = search_by_filters(gu_list=["마포구"], area_center=59, tol=2.0, size=500)
    rows = [r for r in base if "공덕" in str(r.get("dong","")) and "래미안" in str(r.get("apt_name",""))]
    pretty(rows[:30])


{ gu: 마포구, dong: 신공덕동, apt_name: 신공덕래미안2차, deal_date: 2025-10-23, area_m2: 59.5, year_built: 2000, price_krw: 1,400,000,000원 }
{ gu: 마포구, dong: 공덕동, apt_name: 공덕3삼성래미안, deal_date: 2025-10-19, area_m2: 60.0, year_built: 2004, price_krw: 1,950,000,000원 }
{ gu: 마포구, dong: 신공덕동, apt_name: 신공덕1차삼성래미안1아파트, deal_date: 2025-10-18, area_m2: 59.8, year_built: 2000, price_krw: 1,625,000,000원 }
{ gu: 마포구, dong: 신공덕동, apt_name: 신공덕1차삼성래미안1아파트, deal_date: 2025-10-15, area_m2: 59.8, year_built: 2000, price_krw: 1,710,000,000원 }
{ gu: 마포구, dong: 신공덕동, apt_name: 신공덕래미안3차, deal_date: 2025-10-14, area_m2: 60.0, year_built: 2003, price_krw: 1,550,000,000원 }
{ gu: 마포구, dong: 신공덕동, apt_name: 신공덕래미안3차, deal_date: 2025-10-14, area_m2: 60.0, year_built: 2003, price_krw: 1,590,000,000원 }
{ gu: 마포구, dong: 신공덕동, apt_name: 신공덕래미안2차, deal_date: 2025-10-02, area_m2: 59.5, year_built: 2000, price_krw: 1,440,000,000원 }
{ gu: 마포구, dong: 공덕동, apt_name: 공덕1삼성래미안, deal_date: 2025-10-02, area_m2: 59.7, year_built: 1999, pr

In [13]:
avg, n = avg_price_last_3m("송파구", area_center=84, tol=1.5); print(avg, n)
avg_d, n_d = avg_price_last_3m_dong("송파구", "신천동", area_center=84, tol=1.5); print(avg_d, n_d)


2079602691.2181304 353
2845769230.769231 26


In [15]:
!pip -q install langchain langchain-core langchain-community


In [12]:
from langchain_core.tools import tool

@tool("HybridSearch")
def t_hybrid(query: str) -> list:
    """자연어 질의로 실거래 하이브리드 검색을 수행하고 상위 결과 리스트를 반환합니다.
    Args:
        query: 사용자의 한국어 질의 (예: '서울 10억 이하 59형 자이')
    Returns:
        list[dict]: gu, dong, apt_name, deal_date, area_m2, price_krw 등 필드를 포함한 결과
    """
    return search_demo(
        query,
        gu_list=["강서구", "은평구", "서대문구", "마포구", "송파구"],
        area_center=None, tol=1.2, brand_required=False, size=10
    )

@tool("AvgPriceGu")
def t_avg_gu(gu: str, area: float = 59.0) -> dict:
    """특정 '구'의 면적대(area±tol)에 대한 '최근 3개월 평균 실거래가'를 반환합니다.
    Args:
        gu: 예) '송파구'
        area: 중심 면적(㎡). 기본 59
    Returns:
        dict: {'gu', 'area', 'avg', 'n'}
    """
    avg, n = avg_price_last_3m(gu, area_center=area)
    return {"gu": gu, "area": area, "avg": None if avg is None else int(avg), "n": n}

@tool("AvgPriceDong")
def t_avg_dong(gu: str, dong: str, area: float = 59.0) -> dict:
    """특정 '구+동'의 면적대(area±tol)에 대한 '최근 3개월 평균 실거래가'를 반환합니다.
    Args:
        gu: 예) '송파구'
        dong: 예) '신천동'
        area: 중심 면적(㎡). 기본 59
    Returns:
        dict: {'gu','dong','area','avg','n'}
    """
    avg, n = avg_price_last_3m_dong(gu, dong, area_center=area)
    return {"gu": gu, "dong": dong, "area": area, "avg": None if avg is None else int(avg), "n": n}

@tool("Loan")
def t_loan(loan_principal_krw: int, annual_rate_pct: float, years: int) -> dict:
    """원리금균등 방식 월 상환액을 계산합니다.
    Args:
        loan_principal_krw: 대출 원금(원)
        annual_rate_pct: 연이율(%)
        years: 상환 기간(년)
    Returns:
        dict: {'monthly_payment','months','total_payment'}
    """
    return loan_payment(loan_principal_krw, annual_rate_pct, years)


In [16]:
# === 프리셋 유틸 (간단 포맷터 + 원리금균등 상환기) ===
def pretty(rows, cols=("gu","dong","apt_name","deal_date","area_m2","year_built","price_krw")):
    for r in rows:
        line=[]
        for c in cols:
            v=r.get(c)
            if c=="price_krw" and v is not None: v=f"{int(v):,}원"
            if c=="area_m2"  and v is not None: v=f"{float(v):.1f}"
            line.append(f"{c}: {v}")
        print("{ "+", ".join(line)+" }")
    print(f"\n총 {len(rows)}건\n")

def loan_payment(principal_krw: int, annual_rate_pct: float, years: int) -> dict:
    """원리금균등 월 상환액 계산"""
    n = int(years)*12
    r = (annual_rate_pct/100)/12
    pay = principal_krw/n if r==0 else principal_krw*(r*(1+r)**n)/((1+r)**n-1)
    return {"monthly_payment": int(round(pay)), "months": n, "total_payment": int(round(pay*n))}

# === 시연 ①: 서울 10억 이하 59형 '자이' 실거래 ===
print("=== ① 서울 10억 이하 59형 자이 실거래가 ===")
q1 = search_demo(
    "자이 59",
    gu_list=["강서구","은평구","서대문구","마포구","송파구"],
    area_center=59, tol=2.0,
    brand_required=False,     # 쿼리에 브랜드가 있으면 필수로 강제
    price_max=1_500_000_000, # 10억
    size=15
)
pretty(q1)

# === 시연 ②: 송파구 신천동 84㎡ 최근 3개월 평균 ===
print("=== ② 송파구 신천동 84㎡ 최근 3개월 평균 ===")
avg, n = avg_price_last_3m_dong("송파구", "신천동", area_center=84, tol=1.5)
avg_txt = "데이터 없음" if avg is None else f"{int(avg):,}원 (표본 {n}건)"
print("신천동 84㎡ 최근 3개월 평균:", avg_txt, "\n")

# === 시연 ③: 상환 시뮬레이션 (현금 2억, 매매 9.5억, 30년 3.8%) ===
print("=== ③ 상환 시뮬레이션 (현금 2억, 매매 9.5억, 30년 3.8%) ===")
price = 950_000_000
cash  = 200_000_000
loan  = price - cash
res   = loan_payment(loan, 3.8, 30)
print(f"대출원금: {loan:,}원 | 월 상환: {res['monthly_payment']:,}원 | 총 {res['months']}개월\n")


=== ① 서울 10억 이하 59형 자이 실거래가 ===
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2025-09-02, area_m2: 59.8, year_built: 1999, price_krw: 1,150,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2025-06-28, area_m2: 59.8, year_built: 1999, price_krw: 1,048,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2025-05-31, area_m2: 59.8, year_built: 1999, price_krw: 1,095,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2025-05-26, area_m2: 59.8, year_built: 1999, price_krw: 1,035,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2025-03-10, area_m2: 59.8, year_built: 1999, price_krw: 1,040,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2024-12-14, area_m2: 59.8, year_built: 1999, price_krw: 1,035,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2024-12-05, area_m2: 59.8, year_built: 1999, price_krw: 1,030,000,000원 }
{ gu: 송파구, dong: 문정동, apt_name: 문정대우푸르지오1차, deal_date: 2024-07-21, area_